In [1]:
import json
import pandas as pd
import os
import src.modules.helpers as helpers
import src.modules.dtypes as dtypes

list_data = []
periodos_temporada = ['09-10',
                      '10-11',
                      '11-12',
                      '12-13',
                      '13-14',
                      '14-15',
                      '15-16',
                      '16-17',
                      '17-18',
                      '18-19']

path = r"input\dataset"
contenido = os.listdir(path)

for i,archivo in enumerate(contenido):
    path2 = path+'\\'+archivo
    data = json.load(open(path2))
    
    for x in data:     
        #periodo
        x['SeasonPeriod'] = periodos_temporada[i]
        list_data.append(x)
    
df = pd.DataFrame.from_dict(list_data, orient='columns')

df_clean = helpers.clean_df(df, dtypes.game)
df_clean = df_clean.astype(dtypes.game)

del df

In [4]:
#1. obtener todos los campos del equipo local
df_equipo_local = df_clean[['SeasonPeriod','Date','HomeTeam','HC','HF','HR','HS','HST','HY','HTHG','FTHG','FTR']]

#homologar nombres de columnas
df_equipo_local = df_equipo_local.rename(columns={'HomeTeam':'Name',
                                                   'HF':'Faults',
                                                   'HC':'Cards',
                                                   'HR':'CardsRed',
                                                   'HY': 'CardsYellow',
                                                   'HS': 'Shot',
                                                   'HST': 'ShotTarget',
                                                   'HTHG': 'HalfTimeGoals',
                                                   'FTHG': 'FullTimeGoals'
                                                   })

#agregar tipo de equipo: local o visita
df_equipo_local['Type'] = 'Home'

#agregar resultado del juego
df_equipo_local['Result'] = df_equipo_local['FTR'].apply(lambda x: 'Win' if x == 'H' else ('Lose' if x == 'A' else 'Draw') )


In [5]:
#2. obtener todos los campos del equipo visita
df_equipo_visita = df_clean[['SeasonPeriod','Date','AwayTeam','AC','AF','AR','AS','AST','AY','FTAG','HTAG','FTR']]

#homologar nombres de columnas
df_equipo_visita = df_equipo_visita.rename(columns={'AwayTeam':'Name',
                                                   'AF':'Faults',
                                                   'AC':'Cards',
                                                   'AR':'CardsRed',
                                                   'AY': 'CardsYellow',
                                                   'AS': 'Shot',
                                                   'AST': 'ShotTarget',
                                                   'HTAG': 'HalfTimeGoals',
                                                   'FTAG': 'FullTimeGoals'
                                                   })

#agregar tipo de equipo: local o visita
df_equipo_visita['Type'] = 'Away'

#agregar resultado del juego
df_equipo_visita['Result'] = df_equipo_visita['FTR'].apply(lambda x: 'Win' if x == 'A' else ('Lose' if x == 'H' else 'Draw') )

In [6]:
#3. unir df local y visita y generar un solo df: df_equipos
df_equipos = pd.concat([df_equipo_local,df_equipo_visita])
#eliminar df que ya no se van a usar
del df_equipo_local,df_equipo_visita

In [7]:
#4. crear df partido
df_partido = df_clean[['SeasonPeriod','Date','Div','Referee','AwayTeam','HomeTeam','FTR','HTR']]

#cambiar nombres de columnas
df_partido = df_partido.rename(columns={'FTR':'FullTimeResult',
                                        'HTR':'HalfTimeResult'
                                        })
#eliminar df que ya no se van a usar
del df_clean

In [8]:
#5 df principal con season y equipo
df_season_club = df_equipos[['SeasonPeriod', 'Name']].drop_duplicates()

In [9]:
#6. Crear df con resultados

#agrupar y contar los datos por resultado: Win, Lose y Draw
df_partidos_resultados = df_equipos.groupby(
                                            ['SeasonPeriod', 'Name','Result']
                                           ).size().reset_index(name='Count')

#trasponer resultados anteriores, y pasar de filas a columnas, luego se cambian los nombres de columnas, para el reporte
df_partidos_resultados = df_partidos_resultados.pivot_table(
                                                            'Count', ['SeasonPeriod', 'Name'], 'Result'
                                                           ).rename(columns={'Win':'Won',
                                                                             'Lose':'Lost',
                                                                             'Draw':'Drawn'
                                                                            }
                                                                   )

#crear nuevo campo calculado: Played, con la suma de los 3 resultados
df_partidos_resultados['Played'] = df_partidos_resultados['Won']+df_partidos_resultados['Lost']+df_partidos_resultados['Drawn']

In [10]:
#7 crear df con goles a favor

#agrupar datos por periodo y equipo y sumar los goles, luego se modifica el nombre del campo, para el reporte
df_goles_favor = df_equipos.groupby(
                                   ['SeasonPeriod', 'Name'], as_index=False
                                   )['FullTimeGoals'].sum().rename(columns={'FullTimeGoals':'GF'})

In [11]:
#8 crear df con goles en contra

'''
trikiñuela para ir a buscar la suma de goles que me hicieron los equipos con los que jugué,
a: tomando los equipos locales
b.tomando los equipos de visita
'''
#a. a partir de los equipos locales, se busca la suma de los goles que me hicieron los equipos de visita
df_goles_contra_hometeam = (df_partido.merge(
                                            df_equipos,how='inner',
                                            left_on=['SeasonPeriod','Date','AwayTeam'],
                                            right_on=['SeasonPeriod','Date','Name'])
                                            ).groupby(
                                                     ['SeasonPeriod', 'HomeTeam'], as_index=False
                                                     )['FullTimeGoals'].sum().rename(columns={'HomeTeam':'Name'}
                                                                                                                                )
#b. a partir de los equipos visitas, se busca la suma de los goles que me hicieron los equipos locales
df_goles_contra_awayteam =(df_partido.merge(
                                            df_equipos,how='inner',
                                            left_on=['SeasonPeriod','Date','HomeTeam'],
                                            right_on=['SeasonPeriod','Date','Name'])
                                            ).groupby(
                                                     ['SeasonPeriod', 'AwayTeam'], as_index=False
                                                    )['FullTimeGoals'].sum().rename(columns={'AwayTeam':'Name'})

# ahora juntar y sumar todo: a+b

#juntar a y b
df_goles_contra = pd.concat(
                         [df_goles_contra_hometeam,df_goles_contra_awayteam]
                           ).rename(columns={'FullTimeGoals':'GA'})

#sumar los valores, para que quede una sola fila por equipo
df_goles_contra = df_goles_contra.groupby(
                                         ['SeasonPeriod', 'Name'], as_index=False
                                         )['GA'].sum()

#eliminar df que ya no se van a usar
del df_goles_contra_hometeam,df_goles_contra_awayteam

In [12]:
#9.Tabla de posiciones, ahora cruzar todo lo anterior

#a.resultados juego
df_aux1 = df_season_club.merge(
                              df_partidos_resultados,how='inner',on=['SeasonPeriod','Name']
                              )

#b.goles a favor
df_aux2 = df_aux1.merge(
                        df_goles_favor, how='inner', on=['SeasonPeriod','Name']
                        )

#c.goles en contra 
df_tabla_posiciones = df_aux2.merge(
                                   df_goles_contra, how='inner', on=['SeasonPeriod','Name']
                                    )

#d.diferencia de goles
df_tabla_posiciones['GD'] = df_tabla_posiciones['GF'] - df_tabla_posiciones['GA']

#e.puntos
df_tabla_posiciones['Points'] = (df_tabla_posiciones['Won'] * 3)+(df_tabla_posiciones['Drawn'] * 1)

#ordenar
df_tabla_posiciones = df_tabla_posiciones.sort_values(
                                                      ['SeasonPeriod', 'Points'], ascending=[True, False]
                                                     )

#agregar numero de posicion
df_tabla_posiciones['Position'] = df_tabla_posiciones.groupby(
                                                             ['SeasonPeriod']
                                                              ).cumcount()+1

#mostrar columnas ordenadas
df_tabla_posiciones = df_tabla_posiciones[['Position',
                                           'SeasonPeriod',
                                           'Name',
                                           'Played',
                                           'Won',
                                           'Lost',
                                           'Drawn',
                                           'GF',
                                           'GA',
                                           'GD',
                                           'Points']]

#eliminar df que ya no se van a usar
del df_aux1,df_aux2,df_season_club,df_partidos_resultados,df_goles_favor,df_goles_contra

In [13]:
#10. Equipo con mejor relacion de disparos al arco

#obtener suma de tiros al arco y de goles por temporada y equipo
df_aux1 = df_equipos.groupby(
                            ['SeasonPeriod','Name'], as_index=False
                            ).agg(
                                 {
                                  'FullTimeGoals':'sum',
                                  'ShotTarget':'sum'
                                 })

#crear nuevo campo, con la división/relacion de campos anteriores
df_aux1['ShotsGoals'] = df_aux1['FullTimeGoals']/df_aux1['ShotTarget']

#agrupar por periodo y sacar el maximo valor de relacion anterior, luego se obtiene el equipo asociado a ese máximo
df_equipos_mejor_relacion_arco = df_aux1.groupby(
                                                'SeasonPeriod', as_index=False
                                                )['ShotsGoals'].agg(
                                                'max').merge(
                                                            df_aux1, how='inner', on=['SeasonPeriod','ShotsGoals']
                                                            )[['SeasonPeriod','Name','ShotsGoals']]

#eliminar df que ya no se van a usar
del df_aux1

In [14]:
#11. Equipo mas goleado

#agrupar por periodo y sacar el maximo valor de los goles en contra, luego se obtiene el equipo asociado a ese maximo
df_equipo_mas_goleado = df_tabla_posiciones.groupby(
                                                    'SeasonPeriod', as_index=False)['GA'].agg(
                                                     'max').merge(
                                                                  df_tabla_posiciones, how='inner', on=['SeasonPeriod','GA']
                                                                 )[['SeasonPeriod','Name','GA']]

In [16]:
#12. crear archivos de salida csv

#1.- Tabla de posiciones
path_salida = r'output\tabla_posiciones.csv'

df_tabla_posiciones.to_csv(path_salida, sep=';', index = False)

#2.- Equipo con mejor relacion de disparos al arco
path_salida = r'output\equipo_mejor_relacion_disparos_arco.csv'

df_equipos_mejor_relacion_arco.to_csv(path_salida, sep=';', index = False)

#3.- Equipo más goleado
path_salida = r'output\equipo_mas_goleado.csv'

df_equipo_mas_goleado.to_csv(path_salida, sep=';', index = False)

#eliminar df que ya no se van a usar
del df_tabla_posiciones,df_equipos_mejor_relacion_arco,df_equipo_mas_goleado